In [ ]:
# baseado no tutorial: https://keras.io/examples/nlp/pretrained_word_embeddings/

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras import Input, Model
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, GlobalMaxPooling1D

import numpy as np
import os
import codecs
import gensim

from sklearn.model_selection import train_test_split

In [ ]:
# va na aula 
# faça o upload do arquivo FakeCorpus.zip e execute
!unzip 'FakeCorpus.zip' 

In [ ]:
MAX_SEQUENCE_LENGTH = 200
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 300

# Carregando os dados

In [ ]:
def load_data(data_dir, class_label):

    file_names = []
    for (dirpath, dirnames, filenames) in os.walk(data_dir):
        for f in filenames:
            if f.endswith(".txt"):
                file_names.append(os.path.join(dirpath, f))

    txt_ = []
    y = []
    for f in file_names: 
        txt_.append(codecs.open(f,'r',encoding='iso8859-1').read())
        y.append(class_label)

    return txt_, y

fake_data, y_fake = load_data('Fake.Br Corpus/full_texts/fake/', 0)    
true_data, y_true = load_data('Fake.Br Corpus/full_texts/true/', 1)



In [ ]:
data_train, data_test, y_train, y_test = train_test_split(fake_data + true_data, y_fake + y_true,test_size=0.15, random_state=1447)

# Carregando o modelo e processando

In [ ]:
# aqui vamos carregar o modelo, que usamos na aula 03 e em outras aulas, usando o gensim
embedding_model = gensim.models.KeyedVectors.load_word2vec_format("vectors-wikipt.bin", binary=True)

In [ ]:
# primeiro eu tenho que especificar um tokenizador com o vocabulario 
# do modelo pré-treinado. Daí ele vai saber como tokenizar um texto novo recebido.
# Após isso, eu preciso pegar a matriz de embeddings deste modelo
# Com essa matriz, eu já inicializo minha camada de Embeddings do Keras e ela 
# já vai estar inicializada com bons valores para representar minha palavras.
# Por isso o resultado, em geral, com este tipo de estratégia, costuma ser melhor 
# do que depender apenas do treino da camada Embddings do keras
def get_tokenizer(embedding_model):

    # estou pegando todas as minhas palavras em um formato de lista
    all_words = [ w for (w, v) in embedding_model.vocab.items()]

    # mapeando minhas palavras do modelo pré-treinado para um índice
    # inteiro. Esse índice que representará minha palavra w
    vocabulary = {}
    for idx, w in enumerate(all_words):
        vocabulary[w] = idx + 1

    # Inicializando meu tokenizador com o vocabulário já existente
    # no meu modelo pré-treinado
    tokenizer = Tokenizer(len(vocabulary))
    # além disso, eu preciso especificar o mapeamento palavra -> índice
    # Assim o keras vai saber localizar a palavra na matriz
    # Por exemplo, palavra "casa" é índice 10. Então o keras vai na linha 10
    # da matriz e pega a representação daquela palavra
    # O tokenizador que faz essa "ponte"
    tokenizer.word_index = vocabulary

    # definindo uma matriz de pesos com tamanho (numero_palavras + 1) x tamanho do embedding
    # inicialmente essa matriz estará toda com zeros
    weight_matrix = np.zeros((len(all_words) + 1, embedding_model.vector_size))
    # percorrer as palavras e armazená-las na matriz. Fazendo o mapeamento 
    # indice (usado no tokenizador) -> vetor embedding no modelo pré-treinado
    for i in range(len(all_words)):
        weight_matrix[i + 1] = embedding_model[all_words[i]]
    
    return tokenizer, weight_matrix

# Construir e treinar o nosso modelo

In [ ]:
tokenizer, embedding_matrix = get_tokenizer(embedding_model)
# a vetorizacao dos dados, mas feito de outras forma, i.e., usando o tokenizador
X_train = tokenizer.texts_to_sequences(data_train) 
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)

X_test = tokenizer.texts_to_sequences(data_test) 
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
word_index = tokenizer.word_index

# inicializando a nossa camada Embedding com a matriz de pesos e outras informações
# argumento 1: dimensão da entrada, que é o tamanho do nosso vocabulário + 1
# argumento 2: tamanho da "saída" (que será o nosso vetor embedding)
# weights: fizando a matriz de pesos
# input_length: tamanho maximo da sequencia que entrará na rede
# trainable: False evita que os pesos dessa camada sejam atualizados, como já inicializamos com uma matriz de pesos
# que já foi treinada, então não precisamos modificá-la
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
# construindo a arquitetura do nosso modelo

sequence_input = Input(shape=(None,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
#x = Conv1D(128, 5, activation='relu')(x)
#x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(1, activation='softmax')(x)

#learning_rate = 0.00005
learning_rate=0.001
#learning_rate = 0.005
#learning_rate=0.01
#learning_rate=0.05

model = Model(sequence_input, preds)
opt = keras.optimizers.Adam(learning_rate=learning_rate)
#opt = keras.optimizers.Adagrad(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['acc'])
print(model.summary())

In [ ]:
model.fit(X_train, np.array(y_train), epochs=3, batch_size=128)